In [7]:
import pandas as pd
import requests
import hvplot.pandas
import folium

In [2]:
url = 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_week.geojson'
response = requests.get(url)
data = response.json()

In [4]:
earthquake_features = data["features"]
earthquake_data = []
for feature in earthquake_features:
    props = feature['properties']
    coords = feature['geometry']['coordinates']
    earthquake_data.append({
        'magnitude': props['mag'],
        'location': props['place'],
        'time': pd.to_datetime(props['time'], unit='ms'),
        'longitude': coords[0],
        'latitude': coords[1],
        'depth': coords[2]
    })

In [5]:
earthquake_df = pd.DataFrame(earthquake_data)

In [9]:
map_center = [earthquake_df['latitude'].mean(), earthquake_df['longitude'].mean()]
m = folium.Map(location=map_center, zoom_start=2)


In [15]:
def depth_color(depth):
    if depth < 10:
        return 'green'
    elif 10 <= depth < 30:
        return 'yellow'
    elif 30 <= depth < 70:
        return 'orange'
    else:
        return 'red'
    
for index, row in earthquake_df.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=row['magnitude'] * 2,
        color=depth_color(row['depth']),
        fill=True,
        fill_color=depth_color(row['depth']),
        fill_opacity=0.7,
        popup=f"Magnitude: {row['magnitude']}, Depth: {row['depth']} km, Location: {row['location']}"
    ).add_to(m)

m.save('earthquake_map.html')

In [16]:
legend_html = '''
<div style="position: fixed; 
    bottom: 50px; left: 50px; width: 200px; height: 150px; 
    background-color: white; border:2px solid grey; z-index:9999; font-size:14px;
    ">&nbsp; <b>Earthquake Legend</b> <br>
    &nbsp; <i style="background:green"></i>&nbsp; Depth < 10 km <br>
    &nbsp; <i style="background:yellow"></i>&nbsp; 10 km ≤ Depth < 30 km <br>
    &nbsp; <i style="background:orange"></i>&nbsp; 30 km ≤ Depth < 70 km <br>
    &nbsp; <i style="background:red"></i>&nbsp; Depth ≥ 70 km <br>
    <br>
    &nbsp; <b>Marker size</b>: Magnitude
</div> '''

m.get_root().html.add_child(folium.Element(legend_html))

m.save('earthquake_map_with_legend.html')